In [1]:
from __future__ import annotations

import pandas as pd

from src.data_processors_lib.simple_processors import SimpleGroup, SimpleGroupParams
from src.eventstream.schema import RawDataSchema
from src.graph.p_graph import PGraph, EventsNode, Eventstream, MergeNode

source_df = pd.DataFrame([
    {"event_name": "pageview",
     "event_timestamp": "2021-10-26 12:00", "user_id": "1"},
    {"event_name": "cart_btn_click",
     "event_timestamp": "2021-10-26 12:02", "user_id": "1"},
    {"event_name": "pageview",
     "event_timestamp": "2021-10-26 12:03", "user_id": "1"},
    {"event_name": "trash_event",
     "event_timestamp": "2021-10-26 12:03", "user_id": "1"},
    {"event_name": "exit_btn_click",
     "event_timestamp": "2021-10-26 12:04", "user_id": "2"},
    {"event_name": "plus_icon_click",
     "event_timestamp": "2021-10-26 12:05", "user_id": "1"},
])

source = Eventstream(
    raw_data=source_df,
    raw_data_schema=RawDataSchema(
        event_name="event_name",
        event_timestamp="event_timestamp",
        user_id="user_id"
    )
)

cart_events = EventsNode(
    SimpleGroup(SimpleGroupParams(**{
        "event_name": "add_to_cart",
        "filter": lambda df, schema: df[schema.event_name].isin(
            ["cart_btn_click", "plus_icon_click"])
    }))
)
logout_events = EventsNode(
    SimpleGroup(SimpleGroupParams(**{
        "event_name": "logout",
        "filter": lambda df, schema: df[schema.event_name] == "exit_btn_click"
    }))
)
merge = MergeNode()

graph = PGraph(source)
graph.add_node(
    node=cart_events,
    parents=[graph.root]
)
graph.add_node(
    node=logout_events,
    parents=[graph.root]
)
graph.add_node(
    node=merge,
    parents=[
        cart_events,
        logout_events,
    ]
)

result = graph.combine(merge)
result_df = result.to_dataframe()

result_df


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,40c4d942-4fca-4951-bae4-a243c34bd48a,raw,0,pageview,2021-10-26 12:00:00,1
1,35e6e460-9388-4782-8155-1b1a30a88406,group_alias,1,add_to_cart,2021-10-26 12:02:00,1
3,32260373-a1ea-4817-95e7-70d63a8bfe76,raw,3,pageview,2021-10-26 12:03:00,1
4,aec8daaa-32b6-4f8c-b865-91092de187ae,raw,4,trash_event,2021-10-26 12:03:00,1
5,44906780-edf8-4e97-ad64-e2c754af8059,group_alias,5,logout,2021-10-26 12:04:00,2
7,a65c3432-4b55-4720-b4ed-1e1b84eda77c,group_alias,7,add_to_cart,2021-10-26 12:05:00,1


In [2]:
cart_events.processor.params.schema_json()

/tmp/ipykernel_956700/1051905014.py:1: UserWarning: Callable filter was excluded from schema since JSON schema has no equivalent type.
  cart_events.processor.params.schema_json()


'{"title": "SimpleGroupParams", "type": "object", "properties": {"event_name": {"title": "Event Name", "type": "string"}, "event_type": {"title": "Event Type", "default": "group_alias", "type": "string"}}, "required": ["event_name"]}'

In [3]:
# from __future__ import annotations
#
# from src.data_processor.data_processor import DataProcessor, ParamsModel
# from typing import Optional
#
# class CustomParams(ParamsModel):
#     name: str
#     last_name: Optional[str]
#     tags: list[str]
#     numbers: list[int] | str
#
#
# class CustomProcessor(DataProcessor):
#     params: CustomParams
#
#     def __init__(self, params: CustomParams) -> None:
#         super().__init__(params=params)
#
#     def apply(self, eventstream: Eventstream) -> Eventstream:
#         return eventstream
#
#     def schema(self) -> dict[str, dict | str]:
#         schema = {
#             "params": self.params.schema(),
#             "values": self.params.dict()
#         }
#         return schema

In [4]:
dir(graph._PGraph__ngraph)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adj',
 '_node',
 '_pred',
 '_succ',
 'add_edge',
 'add_edges_from',
 'add_node',
 'add_nodes_from',
 'add_weighted_edges_from',
 'adj',
 'adjacency',
 'adjlist_inner_dict_factory',
 'adjlist_outer_dict_factory',
 'clear',
 'clear_edges',
 'copy',
 'degree',
 'edge_attr_dict_factory',
 'edge_subgraph',
 'edges',
 'get_edge_data',
 'graph',
 'graph_attr_dict_factory',
 'has_edge',
 'has_node',
 'has_predecessor',
 'has_successor',
 'in_degree',
 'in_edges',
 'is_directed',
 'is_multigraph',
 'name',
 'nbunch_iter',
 'neighbors',
 'node_attr_dict_factory',
 'node_di

In [5]:
graph._PGraph__ngraph.nodes.data()

NodeDataView({{'name': 'SourceNode', 'pk': 'eb827f5a-f29d-429b-aceb-278f114ff0e6'}: {}, {'name': 'EventsNode', 'pk': '9ff852ef-5496-4153-b3b4-695406d98759'}: {}, {'name': 'EventsNode', 'pk': '85d4397c-6f43-4eec-818c-94d2d129e85c'}: {}, {'name': 'MergeNode', 'pk': '607ebbdb-6367-4e1c-9a92-2409cd268339'}: {}})

In [6]:
import networkx as nx
from itertools import chain


def node_link_data(
        G,
        attrs=None,
        *,
        source="source",
        target="target",
        name="id",
        key="key",
        link="links",
):
    if attrs is not None:
        import warnings

        msg = (
            "\n\nThe `attrs` keyword argument of node_link_data is deprecated\n"
            "and will be removed in networkx 3.2. It is replaced with explicit\n"
            "keyword arguments: `source`, `target`, `name`, `key` and `link`.\n"
            "To make this warning go away, and ensure usage is forward\n"
            "compatible, replace `attrs` with the keywords. "
            "For example:\n\n"
            "   >>> node_link_data(G, attrs={'target': 'foo', 'name': 'bar'})\n\n"
            "should instead be written as\n\n"
            "   >>> node_link_data(G, target='foo', name='bar')\n\n"
            "in networkx 3.2.\n"
            "The default values of the keywords will not change.\n"
        )
        warnings.warn(msg, DeprecationWarning, stacklevel=2)

        source = attrs.get("source", "source")
        target = attrs.get("target", "target")
        name = attrs.get("name", "name")
        key = attrs.get("key", "key")
        link = attrs.get("link", "links")
    # -------------------------------------------------- #
    multigraph = G.is_multigraph()

    # Allow 'key' to be omitted from attrs if the graph is not a multigraph.
    key = None if not multigraph else key
    if len({source, target, key}) < 3:
        raise nx.NetworkXError("Attribute names are not unique.")
    data = {
        "directed": G.is_directed(),
        # "multigraph": multigraph,
        # "graph": G.graph,
        "nodes": [
            dict(
                chain(
                    G.nodes[n].items(),
                    # [(name, n)]
                    [(name, n.export())]
                )
            ) for n in G],
    }
    if multigraph:
        data[link] = [
            dict(chain(d.items(), [(source, u), (target, v), (key, k)]))
            for u, v, k, d in G.edges(keys=True, data=True)
        ]
    else:
        data[link] = [
            dict(chain(d.items(), [(source, u), (target, v)]))
            for u, v, d in G.edges(data=True)
        ]
    return data


node_link_data(graph._PGraph__ngraph)

{'directed': True,
 'nodes': [{'id': {'name': 'SourceNode',
    'pk': 'eb827f5a-f29d-429b-aceb-278f114ff0e6'}},
  {'id': {'name': 'EventsNode',
    'pk': '9ff852ef-5496-4153-b3b4-695406d98759',
    'processor': "{'schema': {'title': 'SimpleGroupParams', 'type': 'object', 'properties': {'event_name': {'title': 'Event Name', 'type': 'string'}, 'event_type': {'title': 'Event Type', 'default': 'group_alias', 'type': 'string'}}, 'required': ['event_name']}, 'values': {'event_name': 'add_to_cart', 'filter': <function <lambda> at 0x7f7cd447e040>, 'event_type': 'group_alias'}}"}},
  {'id': {'name': 'EventsNode',
    'pk': '85d4397c-6f43-4eec-818c-94d2d129e85c',
    'processor': "{'schema': {'title': 'SimpleGroupParams', 'type': 'object', 'properties': {'event_name': {'title': 'Event Name', 'type': 'string'}, 'event_type': {'title': 'Event Type', 'default': 'group_alias', 'type': 'string'}}, 'required': ['event_name']}, 'values': {'event_name': 'logout', 'filter': <function <lambda> at 0x7f7c82

In [7]:
node_link_data(graph._PGraph__ngraph)

{'directed': True,
 'nodes': [{'id': {'name': 'SourceNode',
    'pk': 'eb827f5a-f29d-429b-aceb-278f114ff0e6'}},
  {'id': {'name': 'EventsNode',
    'pk': '9ff852ef-5496-4153-b3b4-695406d98759',
    'processor': "{'schema': {'title': 'SimpleGroupParams', 'type': 'object', 'properties': {'event_name': {'title': 'Event Name', 'type': 'string'}, 'event_type': {'title': 'Event Type', 'default': 'group_alias', 'type': 'string'}}, 'required': ['event_name']}, 'values': {'event_name': 'add_to_cart', 'filter': <function <lambda> at 0x7f7cd447e040>, 'event_type': 'group_alias'}}"}},
  {'id': {'name': 'EventsNode',
    'pk': '85d4397c-6f43-4eec-818c-94d2d129e85c',
    'processor': "{'schema': {'title': 'SimpleGroupParams', 'type': 'object', 'properties': {'event_name': {'title': 'Event Name', 'type': 'string'}, 'event_type': {'title': 'Event Type', 'default': 'group_alias', 'type': 'string'}}, 'required': ['event_name']}, 'values': {'event_name': 'logout', 'filter': <function <lambda> at 0x7f7c82

In [8]:
g = graph._PGraph__ngraph

nodes = [x for x in g]
# dir(next(iter(g)))
# d = next(iter(g))
d = nodes[1]
# [dict(chain(
#     g.nodes[n].items(), [('name', n)]
# )) for n in g]
# [n for n in g]
g.nodes[d].items()

dict_items([])

In [9]:
d.processor

{'schema': {'title': 'SimpleGroupParams', 'type': 'object', 'properties': {'event_name': {'title': 'Event Name', 'type': 'string'}, 'event_type': {'title': 'Event Type', 'default': 'group_alias', 'type': 'string'}}, 'required': ['event_name']}, 'values': {'event_name': 'add_to_cart', 'filter': <function <lambda> at 0x7f7cd447e040>, 'event_type': 'group_alias'}}